# GO Usage Analytics

### Import libraries 

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
import datetime
from datetime import date
from sqlalchemy import create_engine
from sqlalchemy import engine
import psycopg2 as ps

In [3]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    FilterExpression,
    FilterExpressionList,
    Filter
)

----------------------------------------------------------------------------

### Define period of analysis

In [4]:
quarter = '2022-Q4'
start_quarter = '2022-10-01'
end_quarter = '2022-12-31'
start_quarter_last_year = '2021-10-01'

-------------------------------------------------------------

### Bring authentication information

#### GO API

In [5]:
with open('go_authorization_token.json') as json_file:
    go_authorization_token = json.load(json_file)

#### Google Analytics

In [6]:
load_dotenv()
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

In [7]:
#get GA4_PROPERTY_ID as environment variable
go_ga4_property_id = os.environ.get("GA4_PROPERTY_ID")

#### GO database

In [8]:
GO_TOKEN = os.getenv('GO_TOKEN')
GO_DB_NAME = os.getenv('GO_DB_NAME')
GO_DB_HOST = os.getenv('GO_DB_HOST')
GO_DB_USER = os.getenv('GO_DB_USER')
GO_DB_PASSWORD = os.getenv('GO_DB_PASSWORD')

-------------------------------------------------------------------------

### Read data

#### GO API

In [9]:
def fetchUrl(field):
    return requests.get(field, headers = go_authorization_token).json()

In [10]:
def fetchField(field):
    dict_field = []
    errors = []
    temp_dict = requests.get('https://goadmin.ifrc.org/api/v2/'+field+'/?limit=200/', headers = go_authorization_token).json()
    #temp_dict = requests.get('https://goadmin.ifrc.org/api/v2/'+field+'/?limit=200/').json()
    try:
        while temp_dict['next']:
            dict_field.extend(temp_dict['results'])
            temp_dict = fetchUrl(temp_dict['next'])
        dict_field.extend(temp_dict['results'])
        return pd.DataFrame.from_dict(dict_field)
    except:
        print('Problem accessing the webpage: ', field)
        print('========================')

In [11]:
go_events = fetchField('event')

In [12]:
len(go_events)

5363

In [13]:
go_sitrep = fetchField('situation_report')

In [14]:
go_event_snippet = fetchField('event_snippet')

In [15]:
go_country = fetchField('country')

In [16]:
go_country = go_country[pd.notnull(go_country['iso'])]

In [17]:
go_country = go_country[go_country['record_type_display']=='Country']

In [18]:
go_region = fetchField('region')

#### Google Analytics 

In [19]:
def run_report(property_id):
    """Runs a simple report on a Google Analytics 4 property."""
    # service account secret keys specified in GOOGLE_APPLICATION_CREDENTIALS environment variable and read with the function below
    client = BetaAnalyticsDataClient()

    #almost all variables were renamed from UA to GA4 for our use case
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="pagePath"),
                   Dimension(name="month"),
                   Dimension(name="year"),
                   Dimension(name="countryId"),
                   Dimension(name="country")],
        metrics=[Metric(name="activeUsers"),
                 Metric(name="screenPageViews"),#change here https://www.kevinleary.net/blog/unique-pageviews-google-analytics-4/
                 Metric(name="averageSessionDuration")],
        date_ranges=[DateRange(start_date=start_quarter_last_year, end_date=end_quarter)],
        limit=1000000,
        dimension_filter=FilterExpression(
            or_group=FilterExpressionList(
                expressions=[
                    FilterExpression(
                        filter=Filter(
                            field_name="pagePath",
                            string_filter=Filter.StringFilter(value="/emergencies/",match_type = "BEGINS_WITH"),
                        )
                    ),
                    FilterExpression(
                        filter=Filter(
                            field_name="pagePath",
                            string_filter=Filter.StringFilter(value="/emergency/",match_type = "BEGINS_WITH"),
                        )
                    ),
                ]
            )
        ),
    )

    response = client.run_report(request)
    return response

In [20]:
report = run_report(go_ga4_property_id)

#### GO database

In [21]:
#TODO: Manual set up of database in local machine
%load_ext sql

In [23]:
connection = ps.connect(
    database = GO_DB_NAME,
    user = GO_DB_USER,
    password = GO_DB_PASSWORD,
    host = GO_DB_HOST
)

In [24]:
cursor = connection.cursor()

In [25]:
sites_query = cursor.execute("select * from auth_group")
colnames = [desc[0] for desc in cursor.description]
go_auth_group = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [26]:
sites_query = cursor.execute("select * from auth_group_permissions")
colnames = [desc[0] for desc in cursor.description]
go_auth_group_permission = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [27]:
sites_query = cursor.execute("select * from auth_permission")
colnames = [desc[0] for desc in cursor.description]
go_auth_permission = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [28]:
sites_query = cursor.execute("select * from auth_usr")
colnames = [desc[0] for desc in cursor.description]
go_auth_user = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [29]:
sites_query = cursor.execute("select * from auth_user_groups")
colnames = [desc[0] for desc in cursor.description]
go_auth_user_groups = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [30]:
sites_query = cursor.execute("select * from auth_user_user_permissions")
colnames = [desc[0] for desc in cursor.description]
go_auth_user_user_permissions = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [31]:
sites_query = cursor.execute("select * from api_profile")
colnames = [desc[0] for desc in cursor.description]
go_api_profile = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [32]:
sites_query = cursor.execute("select * from api_country")
colnames = [desc[0] for desc in cursor.description]
go_api_country = pd.DataFrame(cursor.fetchall(), columns = colnames)

In [33]:
cursor.close()
connection.close()

#### Undefined data

In [34]:
dict_location_region = {'Africa':'Africa',
                      'Africa Region':'Africa',
                      'Americas':'Americas',
                      'Americas Region':'Americas',
                      'Anguilla':'(not set)',
                      'Aruba':'(not set)',
                      'Asia Pacific':'Asia Pacific', 
                      'Asia-Pacific Region':'Asia Pacific',
                      'Åland Islands':'Europe',
                      'Bermuda':'(not set)',
                      'Caribbean Netherlands':'(not set)',
                      'Curaçao':'(not set)',
                      'Europe':'Europe', 
                      'Europe Region':'Europe',
                      'French Guiana':'(not set)',
                      'Gibraltar': 'Europe',
                      'Guadeloupe':'(not set)',
                      'Hong Kong':'Asia Pacific',
                      'St. Martin':'(not set)',
                      'Macao':'Asia Pacific',
                      'Martinique':'(not set)',
                      'Mayotte':'(not set)',
                      'Middle East & North Africa':'Middle East & North Africa',
                      'New Caledonia':'(not set)',
                      'Oman':'(not set)',
                      'French Polynesia':'(not set)',
                      'Puerto Rico':'Americas',
                      'Réunion':'(not set)',
                      'Sint Maarten':'(not set)',
                      'St. Barthélemy':'(not set)',
                      'Svalbard & Jan Mayen':'Europe',
                      'Taiwan':'Asia Pacific',
                      'Western Sahara':'(not set)',
                      }  

In [35]:
dict_ns_region = {'British Red Cross':'Europe',
                'Macau Red Cross':'Asia Pacific'}  

---------------------------------------------------------------

### Preprocess data

#### Google Analytics

In [36]:
def format_report(response):
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    output = output.reset_index()
    return output

In [37]:
google_views_quarter = format_report(report)

In [38]:
google_views_quarter['month'] = [pd.to_datetime(y+x, format='%Y%m') for x,y in zip(google_views_quarter['month'],google_views_quarter['year'])]

In [39]:
google_views_quarter['quarter'] = [str(x.year)+'-Q'+str((x.month-1)//3 + 1) for x in google_views_quarter['month']]

In [40]:
google_views_quarter['activeUsers'] = [int(x) for x in google_views_quarter['activeUsers']]

In [41]:
google_views_quarter['screenPageViews'] = [int(x) for x in google_views_quarter['screenPageViews']]

In [42]:
google_views_quarter['averageSessionDuration'] = [float(x) for x in google_views_quarter['averageSessionDuration']]

In [43]:
google_views_quarter = google_views_quarter.merge(go_country[['iso','region']], left_on = 'countryId', right_on = 'iso', how = 'left')

In [44]:
google_views_quarter = google_views_quarter.merge(go_region[['id','region_name']], left_on = 'region', right_on = 'id', how = 'left')

In [45]:
len(google_views_quarter)

6734

In [52]:
def ga_clean_no_location(df):
    if len(df[df['countryId'] == '(not set)']) > 0:
        print('Number of viewers from Google Analytics without location:', df[df['countryId'] == '(not set)']['activeUsers'].iloc[0], '. This viewers are removed from the analysis.')
        df = df[df['countryId'] != '(not set)']
    return df

In [53]:
google_views_quarter = ga_clean_no_location(google_views_quarter)

Number of viewers from Google Analytics without location: 1 . This viewers are removed from the analysis.


In [55]:
assert(len(google_views_quarter[pd.isna(google_views_quarter['countryId'])]) == 0)

In [51]:
#TODO: Manual process: Identify region of countries that have not been assigned in GO and add it to the above displayed dictionary
#google_views_quarter['region_name'] = [dict_location_region[y] if pd.isna(x) else x for x,y in zip(google_viewers_quarter['region_name'],google_viewers_quarter['countryId'])]

In [56]:
def filter_existing_pages_id(google_views_quarter, go_events):
    ''' Filter Google Analytics emergency page pathways that have a connection to the GO collection of emergency pages.
    This involves eliminating Google Analytics data obtained from users who clicked on links that were invalid 
    and only matched the GO dataset's format for emergency pages.
    '''
    
    go_emg_collection = ['/emergencies/'+str(x) for x in go_events['id']]
    go_emg_collection.extend(['/emergency/'+x for x in go_events['slug'] if pd.notnull(x)])
    go_emg_collection.extend('/emergency/'+str(x) for x in go_events['id'])
    
    mask = [x in go_emg_collection for x in google_views_quarter['pagePath']]
    filtered = google_views_quarter[mask]
    removed = pd.concat([google_views_quarter,filtered]).drop_duplicates(keep=False)
    
    nb_pages_out = len(np.setdiff1d(google_views_quarter['pagePath'],go_emg_collection))
    nb_users_pages_out= removed['activeUsers'].sum()
    nb_views_pages_out= removed['screenPageViews'].sum()
    
    print(nb_pages_out,' GO \'emergency pages\' links that are not functioning (possibly because the emergencies/events were deleted), were visited by ',nb_users_pages_out,' active users in ', nb_views_pages_out,'views.') 
    return filtered

In [57]:
google_views_quarter = filter_existing_pages_id(google_views_quarter, go_events)

55  GO 'emergency pages' links that are not functioning (possibly because the emergencies/events were deleted), were visited by  503  active users in  976 views.


In [58]:
google_views_quarter.rename(columns ={'countryId':'Country','country':'CountryName','pagePath':'Page','activeUsers':'# Viewers','screenPageViews':'Views','averageSessionDuration':'Avg Time On Page','month':'Month','quarter':'Date'}, inplace = True)

In [59]:
assert(len(google_views_quarter[google_views_quarter[['Page','Month','Country']].duplicated()])==0)

#### GO API

##### Events

In [60]:
go_events['created_at'] = [pd.to_datetime(x) if pd.notna(x) else np.nan for x in go_events['created_at']]

In [61]:
go_events['updated_at'] = [pd.to_datetime(x) if pd.notna(x) else np.nan for x in go_events['updated_at']]

In [62]:
go_events['Page'] = ['/emergencies/'+str(x) for x in go_events['id']]

In [63]:
go_events['countries_list'] = [[c['society_name'] for c in x] if len(x) > 0 else np.nan for x in go_events['countries']]

In [64]:
len(go_events)

5363

In [65]:
go_events = go_events.explode(column = 'countries_list')

In [66]:
go_events.rename(columns = {'countries_list':'National Society'}, inplace = True)

In [67]:
len(go_events)

5656

In [68]:
go_events['region_id'] = [x[0]['region'] if len(x) > 0 and pd.notnull(x[0]['region']) else np.nan for x in go_events['countries']]

In [69]:
go_events = go_events.merge(go_region[['id','region_name']], left_on = 'region_id', right_on = 'id', how = 'left')

In [70]:
go_events.rename(columns = {'region_name':'Region'}, inplace = True)

In [71]:
#Removal of events without associated location
mask = [bool(x) for x in go_events['countries']]
go_events = go_events[mask]

In [72]:
len(go_events)

5128

In [73]:
go_events['Region'] = [y[0]['name'] if pd.isna(x) else x for x,y in zip(go_events['Region'], go_events['countries'])]

In [74]:
go_events['Region'] = go_events['Region'].map(dict_location_region)

In [75]:
#TODO: Verify all locations are mapped into regions. If not, update dictionary!
assert(len(go_events['Region'].unique()) == 6)

##### Event Snippets

In [76]:
go_event_snippet.rename(columns = {'id':'snippets'}, inplace = True)

##### Situation Reports

In [77]:
go_sitrep['type_report'] = [x['type'] if pd.notnull(x) else np.nan for x in go_sitrep['type']]

In [78]:
go_sitrep['created_at'] = [pd.to_datetime(x) for x in go_sitrep['created_at']]

##### National Societies

In [79]:
mask = [bool(x) for x in go_country['society_name']]
go_ns = go_country[mask]

In [80]:
go_ns = go_ns.merge(go_region[['id','region_name']], left_on = 'region', right_on = 'id', how = 'left')[['region_name','society_name','name']]

In [81]:
go_ns = go_ns.drop_duplicates('society_name')

In [82]:
go_ns = go_ns[go_ns['society_name'] != 'Benelux ERU']

In [83]:
def define_ns_region(df):
    try:
        df['region_name'] = [dict_ns_region[x] if pd.isna(y) else y for x,y in zip(df['society_name'], df['region_name'])]
        return df
    except:
        print('It was not possible to do a mapping between a National Society and a Region')

In [84]:
go_ns = define_ns_region(go_ns)

In [85]:
assert(len(go_ns[pd.isna(go_ns['region_name'])]) == 0)

In [86]:
go_ns_names = go_ns.merge(go_api_country[['society_name_en','society_name_fr','society_name_es','society_name_ar']], how = 'left', left_on = 'society_name', right_on = 'society_name_en').drop_duplicates()

#### GO Database

##### Users

In [87]:
go_users = go_auth_user.merge(go_api_profile, how = 'left', left_on = 'id', right_on = 'user_id')

In [88]:
go_users = go_users.merge(go_country[['id','name','region']], how= 'left', left_on = 'country_id', right_on = 'id')

In [89]:
go_users = go_users.merge(go_region[['id','region_name']], how = 'left', left_on = 'region', right_on = 'id')

In [90]:
go_users.drop(columns = ['id_y','id'], inplace = True)

In [91]:
go_users.rename(columns = {'id_x':'id'}, inplace = True)

In [92]:
go_users['last_frontend_login'] = [pd.to_datetime(x) if pd.notna(x) else np.nan for x in go_users['last_frontend_login']]

In [93]:
go_users['last_login'] = [pd.to_datetime(x) if pd.notna(x) else np.nan for x in go_users['last_login']]

In [94]:
go_users['date_joined'] = [pd.to_datetime(x) if pd.notna(x) else np.nan for x in go_users['date_joined']]

In [95]:
def logged_in_quarter(go_users):
    start_quarter_date = pd.to_datetime(start_quarter, utc = True)
    end_quarter_date = pd.to_datetime(end_quarter, utc = True)
    
    mask_fe_login = [x >= start_quarter_date if pd.notna(x) else False for x in go_users['last_frontend_login']]
    mask_login = [x >= start_quarter_date if pd.notna(x) else False for x in go_users['last_login']]
    mask_joined = [x >= start_quarter_date if pd.notna(x) else False for x in go_users['date_joined']]
    
    mask = [any([a,b,c]) for a,b,c in zip(mask_fe_login, mask_login, mask_joined)]
    return go_users[mask]

In [96]:
go_users_quarter = logged_in_quarter(go_users)

In [97]:
go_users_quarter_ns = go_users_quarter[go_users_quarter['org_type'] == 'NTLS']

In [98]:
go_users_quarter_other = go_users_quarter[go_users_quarter['org_type'] != 'NTLS']

In [99]:
def matched_ns_affiliation(go_users_ns):
    
    ns_en = go_users_ns.merge(go_ns_names[['society_name_en']], left_on = 'org', right_on = 'society_name_en', how = 'inner')
    ns_en.drop(columns = ['society_name_en'], inplace = True)
    
    ns_fr = go_users_ns.merge(go_ns_names[['society_name_en','society_name_fr']], left_on = 'org', right_on = 'society_name_fr', how = 'inner')
    ns_fr['org'] = ns_fr['society_name_en']
    ns_fr.drop(columns = ['society_name_fr','society_name_en'], inplace = True)
    
    ns_es = go_users_ns.merge(go_ns_names[['society_name_en','society_name_es']], left_on = 'org', right_on = 'society_name_es', how = 'inner')
    ns_es['org'] = ns_es['society_name_en']
    ns_es.drop(columns = ['society_name_es','society_name_en'], inplace = True)
    
    ns_ar = go_users_ns.merge(go_ns_names[['society_name_en','society_name_ar']], left_on = 'org', right_on = 'society_name_ar', how = 'inner')
    ns_ar['org'] = ns_ar['society_name_en']
    ns_ar.drop(columns = ['society_name_ar','society_name_en'], inplace = True)
    
    return pd.concat([ns_en, ns_fr, ns_es, ns_ar])

In [100]:
go_users_quarter_ns_part1 = matched_ns_affiliation(go_users_quarter_ns)

In [101]:
# TODO: find those users without a valid national society name in quarter
go_users_quarter_ns_part2 = pd.DataFrame(data = {'id': np.setdiff1d(go_users_quarter_ns['id'], go_users_quarter_ns_part1['id'])}).merge(go_users_quarter_ns, how = 'left')
print('There are ', len(go_users_quarter_ns_part2 ), 'NS users without a valid national society name')
#go_users_quarter_ns_part2

There are  8 NS users without a valid national society name


In [104]:
assert(len(go_users_quarter_ns_part2)==0)

AssertionError: 

In [ ]:
#TODO: clean dataset from admin site OR country NS by default

In [105]:
def infer_ns_affiliation(go_users):
    go_users = go_users.merge(go_ns[['name','society_name']], on = 'name', how = 'left')
    go_users['org'] = go_users['society_name']
    go_users.drop(columns = ['society_name'], inplace = True)
    return go_users

In [106]:
go_users_quarter_ns_part2 = infer_ns_affiliation(go_users_quarter_ns_part2)

In [107]:
assert(len(go_users_quarter_ns_part2) == len(infer_ns_affiliation(go_users_quarter_ns_part2)))

In [108]:
go_users_quarter_ns = pd.concat([go_users_quarter_ns_part1,go_users_quarter_ns_part2])

In [109]:
print('There are ',len(go_users_quarter_other[pd.isna(go_users_quarter_other['name'])]),' users that are not from national societies, whose country location information is not available. These users are removed from the analysis.')
go_users_quarter_other = go_users_quarter_other[pd.notna(go_users_quarter_other['name'])]

There are  25  users that are not from national societies, whose country location information is not available. These users are removed from the analysis.


In [113]:
mask = [x.startswith('country_admin_') for x in go_auth_permission['codename']]
go_country_auth_permission = go_auth_permission[mask].copy()

In [114]:
go_country_auth_permission['country_name'] = [int(x.split('_')[-1]) for x in go_country_auth_permission['codename']]

In [115]:
go_country_auth_permission = go_country_auth_permission.merge(go_country[['id','name','society_name']], left_on = 'country_name', right_on = 'id', how = 'left')

---------------------------------------------------

### Analysis

#### Historical and Geographical Views/Viewers of Emergencies

In [116]:
emergencies = google_views_quarter.merge(go_events[['Page','National Society','Region','name']], on = 'Page', how = 'inner')

In [117]:
emergencies['URL'] = ['https://go.ifrc.org'+x for x in emergencies['Page']]

In [118]:
emergencies['Emergency Name'] = ['['+x.split('/')[-1] + '] '+ y for x,y in zip(emergencies['Page'],emergencies['name'])]

In [119]:
page_1 = emergencies.groupby(['Region','National Society','Page','Emergency Name','Date','Country'], as_index =  False).sum(numeric_only =True)[['Region','National Society','Page','Emergency Name','Date','Country','Views']]

In [120]:
page_1.to_excel('../'+quarter+'/go_usage_regional_pg1.xlsx')

#### Page views in quarter

In [121]:
emergencies_quarter = emergencies[emergencies['Date'] == quarter]

In [122]:
emergencies_quarter = emergencies_quarter.groupby(['Region','National Society','Page','URL','Date'], as_index = False).agg({'# Viewers':'sum','Views':'sum', 'Avg Time On Page':'mean'})

In [123]:
go_events_quarter = go_events[go_events['created_at']>pd.to_datetime(start_quarter, utc =True)]
go_events_quarter = go_events_quarter[go_events_quarter['created_at']<pd.to_datetime(end_quarter, utc =True)]

In [124]:
emg_wo_views = np.setdiff1d(go_events_quarter['Page'].unique(),emergencies_quarter['Page'].unique())

In [125]:
emg_wo_views_df = pd.DataFrame(data = {'Page':emg_wo_views}).merge(go_events_quarter[['Page','National Society','Region']])
emg_wo_views_df['URL'] = ['https://go.ifrc.org' + x for x in emg_wo_views_df['Page']]
emg_wo_views_df['Date'] = quarter
emg_wo_views_df['# Viewers'] = 0
emg_wo_views_df['Views'] = 0
emg_wo_views_df['Avg Time On Page'] = 0

In [126]:
emergencies_quarter = pd.concat([emergencies_quarter,emg_wo_views_df])

In [127]:
page_2 = emergencies_quarter.merge(go_events[['Page','created_at','name']].drop_duplicates(), on = 'Page', how = 'left')

In [128]:
page_2['created_at'] = [x.strftime('%d/%m/%Y') for x in page_2['created_at']]

In [129]:
page_2.to_excel('../'+quarter+'/go_usage_regional_pg2.xlsx')

#### National Society general use of GO

In [130]:
#analyze same Emergency Pages as in page 2
go_events_quarter = page_2[['Page']].merge(go_events, how = 'left')

In [131]:
go_events_quarter['nb_field_reports'] = [len(x) for x in go_events_quarter['field_reports']]

In [132]:
go_events_quarter['nb_appeals'] = [len(x) for x in go_events_quarter['appeals']]

In [133]:
#### Adding information of Situation Reports
# we don't have into account reports submitted after quarter of analysis
go_sitrep_quarter = go_sitrep[go_sitrep['created_at']<pd.to_datetime(end_quarter, utc =True)]
go_sitrep_quarter = go_sitrep_quarter.groupby(['event','type_report'], as_index = False).count()[['event','type_report','id']]
go_sitrep_quarter = go_sitrep_quarter.pivot(index = ['event'], columns = ['type_report'], values = ['id'])
go_sitrep_quarter = go_sitrep_quarter.droplevel(level=0, axis = 1)
go_sitrep_quarter['nb_sitrep'] = go_sitrep_quarter.sum(axis = 1)
go_events_quarter = go_events_quarter.merge(go_sitrep_quarter, left_on = 'id_x', right_on = 'event', how = 'left')

In [134]:
#### Adding information of Event Snippets
# we don't have information of creation date, so we take into account all the snippets related to the emergency page, whether they were created in the quarter of analysis or not [still the emergency should have been active on the quarter of analysis]
go_event_snippet = go_event_snippet[['event','snippets']].groupby('event', as_index = False).count()
go_event_snippet.rename(columns = {'snippets':'nb_snippets'}, inplace = True)
go_events_quarter = go_events_quarter.merge(go_event_snippet, left_on = 'id_x', right_on = 'event', how = 'left')

In [135]:
go_events_quarter['ind_appeals'] = [bool(x) and pd.notna(x) for x in go_events_quarter['nb_appeals']]
go_events_quarter['ind_field_reports'] = [bool(x) and pd.notna(x) for x in go_events_quarter['nb_field_reports']]
go_events_quarter['ind_sitrep'] = [bool(x) and pd.notna(x) for x in go_events_quarter['nb_sitrep']]
go_events_quarter['ind_snippets'] = [bool(x) and pd.notna(x) for x in go_events_quarter['nb_snippets']]

In [136]:
page_3 = go_events_quarter.groupby(['National Society'], as_index = False).agg({'id_x':'count',
                                                                                         'ind_appeals':'mean',
                                                                                         'ind_field_reports':'mean',
                                                                                        'ind_sitrep':'mean',
                                                                                        'ind_snippets':'mean'})

In [137]:
page_3['ind_appeals'] = [np.round(x*100) for x in page_3['ind_appeals']]
page_3['ind_field_reports'] = [np.round(x*100) for x in page_3['ind_field_reports']]
page_3['ind_sitrep'] = [np.round(x*100) for x in page_3['ind_sitrep']]
page_3['ind_snippets'] = [np.round(x*100) for x in page_3['ind_snippets']]

In [138]:
page_3.rename(columns = {'id_x':'# Emergencies',
              'ind_appeals':'% Emergencies with Appeals',
              'ind_field_reports':'% Emergencies with Field Reports',
              'ind_sitrep':'% Emergencies with Situation Reports',
              'ind_snippets':'% Emergencies with Additional Content'}, inplace = True)

In [139]:
#### Adding Country Admin Information
#We don't have temporal information on this. So we give the 'latest' information we have per NS
# We only care for entities related to national societies
go_country_auth_permission = go_country_auth_permission[pd.notna(go_country_auth_permission['society_name'])]

#empty spaces on society name are also geogrpahical entities we are not covering now
mask = [bool(x) for x in go_country_auth_permission['society_name']]
go_country_auth_permission = go_country_auth_permission[mask]
go_country_auth_permission.rename(columns = {'id_x':'permission_id'}, inplace = True)

go_auth_user_user_permissions = go_auth_user_user_permissions.merge(go_country_auth_permission[['permission_id','society_name']], on = 'permission_id')

go_country_admin = go_auth_user_user_permissions.groupby('society_name', as_index = False).count()[['society_name','permission_id']]
go_country_admin =  go_ns.merge(go_country_admin, how = 'left')
go_country_admin.rename(columns = {'society_name':'National Society'}, inplace = True)

page_3 = go_country_admin.merge(page_3, how = 'left')
page_3 = page_3.rename(columns = {'permission_id':'# GO Country Admins'})
page_3 = page_3.rename(columns = {'region_name':'Region'})
page_3 = page_3.fillna(0)
page_3['Quarter of Analysis'] = quarter

In [140]:
page_3.to_excel('../'+quarter+'/go_usage_regional_pg3.xlsx')

#### Users access to GO

In [141]:
#National Society access
page_4a = go_users_quarter_ns.groupby(['region_name','org'], as_index = False).count()[['region_name','org','username']]
page_4a.rename(columns = {'region_name':'Region', 'org':'National Society', 'username':'# Users that logged in'}, inplace = True)
page_4a['Quarter of Analysis'] = quarter
page_4a.to_excel('../'+quarter+'/go_usage_regional_pg4a.xlsx')

In [142]:
#Other users access
page_4b = go_users_quarter_other.groupby(['region_name','org_type'], as_index = False).count()[['region_name','org_type','username']]
page_4b.rename(columns = {'region_name':'Region', 'org_type':'Org Type', 'username':'# Users that logged in'}, inplace = True)
page_4b['Quarter of Analysis'] = quarter
page_4b.to_excel('../'+quarter+'/go_usage_regional_pg4b.xlsx')

In [143]:
#access to GO -locations from Google Analytics
page_4c = google_views_quarter[['region_name','CountryName','# Viewers']].copy()
page_4c.rename(columns = {'region_name': 'Region','CountryName':'Country'}, inplace = True)
page_4c['Quarter of Analysis'] = quarter
page_4c.to_excel('../'+quarter+'/go_usage_regional_pg4c.xlsx')

#### List of regions (to use as a filter)

In [144]:
regions = go_region[['region_name']].copy()
regions.rename(columns = {'region_name':'Region'}, inplace = True)
regions.to_excel('../'+quarter+'/regions.xlsx')

In [145]:
regions.to_excel('../'+quarter+'/regions.xlsx')

---------------------------------------------------------

### Visualization

The output of this process is visualized in [this dashboard](https://app.powerbi.com/view?r=eyJrIjoiY2RlOTRkOGQtMDU5Yy00OWIwLWE2NmYtNTQ5NTQ3YjEwY2ZmIiwidCI6ImEyYjUzYmU1LTczNGUtNGU2Yy1hYjBkLWQxODRmNjBmZDkxNyIsImMiOjh9&pageName=ReportSectione263ecb5066f3105a8fa)